In [56]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer

train_data_path = 'train.tsv'
test_data_path = 'valid.tsv'
output_path = 'output.txt'
stop_path = 'stopwords.txt'

In [57]:
train_df= pd.read_csv(train_data_path,header=None,sep='\t',quoting=3)
train_x= train_df[2]
train_y= train_df[1]
stop= pd.read_csv(stop_path,header=None)[0]
stop=stop.astype(str).tolist()

In [58]:
stemmer = PorterStemmer()

def preprocess_text(text):
    words = text.lower().split()
    filtered_words = [stemmer.stem(word) for word in words if word not in stop]
    return filtered_words

train_x= train_x.apply(preprocess_text)

In [59]:
label_to_index = {'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}
index_to_label = {v: k for k, v in label_to_index.items()}


In [60]:
def create_feature_matrix(processed_texts, labels):
    vocabulary = set(word for text in processed_texts for word in text)
    vocab_size = len(vocabulary)
    
    word_to_index = {word: idx for idx, word in enumerate(vocabulary)}
    
    feature_matrix = np.zeros((len(processed_texts), vocab_size), dtype=int)
    class_count = np.zeros(len(set(labels)))

    # Fill the feature matrix
    for i, text in enumerate(processed_texts):
        for word in text:
            if word in word_to_index:
                feature_matrix[i, word_to_index[word]] += 1 

    # Encode labels
    encoded_labels = np.array([label_to_index[label] for label in labels])
    
    # Calculate class count as the sum of words in sentences of each class
    for i, label in enumerate(encoded_labels):
        class_count[label] += feature_matrix[i].sum()  # Sum of words in the sentence

    return feature_matrix, encoded_labels, class_count, word_to_index


X, Y, class_count, word_to_index= create_feature_matrix(train_x, train_y)

In [61]:
class MultinomialNaiveBayes:
    def __init__(self):
        self.class_priors = None
        self.feature_probs = None
        self.unseen_word_value_per_class = None

    def fit(self, X, y, class_count):
        self.class_priors = np.log(np.bincount(y) / len(y))
        self.feature_probs = np.zeros((len(self.class_priors), X.shape[1]))
        for c in range(len(self.class_priors)):
            X_c = X[y == c]
            self.feature_probs[c] = (X_c.sum(axis=0) + 1) / (X_c.sum() + X.shape[1])
        
        # class_count = np.bincount(y)
        self.unseen_word_value_per_class = [1 / (X.shape[1] + count) for count in class_count]

    def predict(self, X):
        eps = 1e-10 
        log_feature_probs = np.log(self.feature_probs.T.clip(eps, 1))
        log_probs = X @ log_feature_probs + self.class_priors
        print(log_probs)
        return np.argmax(log_probs, axis=1)

    def predict_test(self, X_test, unseen_word_count):
        eps = 1e-10 
        log_feature_probs = np.log(self.feature_probs.T.clip(eps, 1))
        log_probs = X_test @ log_feature_probs + self.class_priors
        for i in range(X_test.shape[0]): 
            for c in range(len(self.class_priors)): 
                if unseen_word_count[i] > 0:
                    unseen_contribution = unseen_word_count[i] * np.log(self.unseen_word_value_per_class[c])
                    log_probs[i, c] += unseen_contribution 
        
        print(log_probs)
        return np.argmax(log_probs, axis=1)


model = MultinomialNaiveBayes()
model.fit(X, Y, class_count)

predictions = model.predict(X)

accuracy = np.mean(predictions == Y)
print(f'Accuracy: {accuracy * 100:.2f}%')

[[ -79.20168302  -73.38481779  -76.42009859  -76.60680196  -79.23488158
   -78.66496927]
 [-130.76901125 -123.2872824  -122.85123868 -120.31100548 -124.14486163
  -125.09011402]
 [-113.60605559 -106.72131004 -105.32299042 -107.04734022 -104.89368197
  -107.74715963]
 ...
 [-141.04162105 -136.90589934 -135.44914128 -131.63397375 -137.79711004
  -136.6408239 ]
 [ -65.44113561  -62.09104149  -64.9220671   -65.77545005  -62.71106984
   -64.70611208]
 [-146.27093672 -150.76535457 -152.69511013 -152.93482643 -154.68076586
  -151.76385062]]
Accuracy: 73.92%


In [62]:
predicted_labels = [index_to_label[pred] for pred in predictions]
print(predicted_labels)
print(len(predicted_labels))
with open(output_path, 'w') as f:
    for label in predicted_labels:
        f.write(f"{label}\n")

['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'barely-true', 'half-true', 'true', 'barely-true', 'half-true', 'half-true', 'barely-true', 'mostly-true', 'mostly-true', 'false', 'barely-true', 'false', 'mostly-true', 'half-true', 'false', 'half-true', 'half-true', 'half-true', 'false', 'mostly-true', 'barely-true', 'true', 'true', 'true', 'pants-fire', 'half-true', 'false', 'half-true', 'false', 'pants-fire', 'false', 'half-true', 'pants-fire', 'false', 'pants-fire', 'false', 'barely-true', 'barely-true', 'false', 'half-true', 'barely-true', 'mostly-true', 'half-true', 'barely-true', 'false', 'mostly-true', 'half-true', 'mostly-true', 'true', 'mostly-true', 'false', 'pants-fire', 'true', 'false', 'barely-true', 'barely-true', 'true', 'half-true', 'mostly-true', 'half-true', 'mostly-true', 'true', 'false', 'barely-true', 'half-true', 'half-true', 'false', 

In [63]:
file= 'checker_files/multinomial_probas_train.npy'
given= np.load(file)
print(np.log(given))
argmax_indices = np.argmax(given, axis=1)
argmax_indices= [index_to_label[z] for z in argmax_indices]
print(argmax_indices)
print(len(argmax_indices))
print(argmax_indices==predicted_labels)

[[ -79.20168302  -73.38481779  -76.42009859  -76.60680196  -79.23488158
   -78.66496927]
 [-130.76901125 -123.2872824  -122.85123868 -120.31100548 -124.14486163
  -125.09011402]
 [-113.60605559 -106.72131004 -105.32299042 -107.04734022 -104.89368197
  -107.74715963]
 ...
 [-141.04162105 -136.90589934 -135.44914128 -131.63397375 -137.79711004
  -136.6408239 ]
 [ -65.44113561  -62.09104149  -64.9220671   -65.77545005  -62.71106984
   -64.70611208]
 [-146.27093672 -150.76535457 -152.69511013 -152.93482643 -154.68076586
  -151.76385062]]
['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'barely-true', 'half-true', 'true', 'barely-true', 'half-true', 'half-true', 'barely-true', 'mostly-true', 'mostly-true', 'false', 'barely-true', 'false', 'mostly-true', 'half-true', 'false', 'half-true', 'half-true', 'half-true', 'false', 'mostly-true', 'barely-true', 'true', 't

In [64]:
test_df= pd.read_csv(test_data_path,header=None,sep='\t',quoting=3)
test_x= test_df[2]
test_y= test_df[1]

In [65]:
def create_test_feature_matrix(processed_texts, word_to_index):
    vocab_size = len(word_to_index)
    
    feature_matrix = np.zeros((len(processed_texts), vocab_size), dtype=int)
    unseen_word_count = np.zeros(len(processed_texts), dtype=int)
    
    for i, text in enumerate(processed_texts):
        for word in text:
            if word in word_to_index:
                feature_matrix[i, word_to_index[word]] += 1  # Count occurrences of each word
            else:
                unseen_word_count[i] += 1
    return feature_matrix, unseen_word_count

test_x= test_x.apply(preprocess_text)
X_test, unseen_word_count = create_test_feature_matrix(test_x, word_to_index)

test_predictions = model.predict_test(X_test, unseen_word_count)
test_predictions = [index_to_label[pred] for pred in test_predictions]
test_accuracy = np.mean(test_predictions == test_y)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


[[ -35.1059684   -33.47698725  -33.09891181  -32.19080153  -31.96328829
   -33.11674228]
 [-123.03648474 -122.9921731  -122.27291767 -122.70606273 -123.61309923
  -121.32514022]
 [-150.67172659 -144.73193404 -146.3430688  -146.18937774 -144.9322682
  -147.59370521]
 ...
 [ -77.27379138  -72.76942319  -73.1945343   -71.53104286  -73.02909389
   -72.2457496 ]
 [-187.09762558 -176.13803383 -177.94828144 -175.91503269 -179.23325708
  -177.60145899]
 [-107.25141964  -99.95253086 -103.25530529 -101.93426204 -102.01042566
  -103.81215491]]
Test Accuracy: 24.69%


In [66]:
print(test_predictions)

['mostly-true', 'true', 'false', 'false', 'half-true', 'half-true', 'mostly-true', 'false', 'false', 'barely-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'barely-true', 'false', 'half-true', 'true', 'mostly-true', 'false', 'barely-true', 'false', 'true', 'half-true', 'half-true', 'half-true', 'false', 'false', 'mostly-true', 'mostly-true', 'half-true', 'false', 'true', 'half-true', 'false', 'mostly-true', 'half-true', 'false', 'half-true', 'half-true', 'barely-true', 'barely-true', 'half-true', 'mostly-true', 'barely-true', 'half-true', 'half-true', 'barely-true', 'barely-true', 'half-true', 'barely-true', 'barely-true', 'half-true', 'barely-true', 'false', 'half-true', 'half-true', 'mostly-true', 'true', 'mostly-true', 'barely-true', 'mostly-true', 'barely-true', 'barely-true', 'barely-true', 'half-true', 'half-true', 'false', 'false', 'half-true', 'half-true', 'half-true', 'half-true', 'mostly-true', 'barely-true', 'false', 'half-true', 'false', 'half-true', 'mostly

In [67]:
print(test_y)

0       barely-true
1        pants-fire
2             false
3         half-true
4         half-true
           ...     
1279      half-true
1280    mostly-true
1281           true
1282          false
1283    barely-true
Name: 1, Length: 1284, dtype: object


In [68]:
valfile= 'checker_files/multinomial_probas_test.npy'
givenval= np.load(valfile)
print(np.log(givenval))
argmax_indices2 = np.argmax(givenval, axis=1)
argmax_indices2= [index_to_label[z] for z in argmax_indices2]
print(argmax_indices2)
print(len(argmax_indices2))
print(argmax_indices2==test_predictions)

[[ -35.1059684   -33.47698725  -33.09891181  -32.19080153  -31.96328829
   -33.11674228]
 [-123.03648474 -122.9921731  -122.27291767 -122.70606273 -123.61309923
  -121.32514022]
 [-150.67172659 -144.73193404 -146.3430688  -146.18937774 -144.9322682
  -147.59370521]
 ...
 [ -77.27379138  -72.76942319  -73.1945343   -71.53104286  -73.02909389
   -72.2457496 ]
 [-187.09762558 -176.13803383 -177.94828144 -175.91503269 -179.23325708
  -177.60145899]
 [-107.25141964  -99.95253086 -103.25530529 -101.93426204 -102.01042566
  -103.81215491]]
['mostly-true', 'true', 'false', 'false', 'half-true', 'half-true', 'mostly-true', 'false', 'false', 'barely-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'barely-true', 'false', 'half-true', 'true', 'mostly-true', 'false', 'barely-true', 'false', 'true', 'half-true', 'half-true', 'half-true', 'false', 'false', 'mostly-true', 'mostly-true', 'half-true', 'false', 'true', 'half-true', 'false', 'mostly-true', 'half-true', 'false', 'half-true',

In [69]:
for i in range(1284):
    if argmax_indices2[i]!=test_predictions[i]:
        print(i)